In [295]:
import pandas as pd
import numpy as np
import plotly.express as px
import re

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

#TextBlob
from textblob import TextBlob

#SpaCy
import spacy
from spacy import displacy

from langdetect import detect

In [296]:
df = pd.read_csv("../data/netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


# Exploratory Analysis and Cleaning

In [297]:
df.shape

(8807, 12)

In [298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [299]:
df.duplicated().sum()

0

In [300]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [301]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
release_year,8807.0,2014.180198,8.819312,1925.0,2013.0,2017.0,2019.0,2021.0


In [302]:
df.describe(include = "object").T

,count,unique,top,freq
show_id,8807,8807,s8302,1
type,8807,2,Movie,6131
title,8807,8807,Wonho Chung: Live in New York,1
director,6173,4528,Rajiv Chilaka,19
cast,7982,7692,David Attenborough,19
country,7976,748,United States,2818
date_added,8797,1767,"January 1, 2020",109
rating,8803,17,TV-MA,3207
duration,8804,220,1 Season,1793
listed_in,8807,514,"Dramas, International Movies",362


## Clean `director`, `cast`, `country`

In [303]:
# remove the null values in "director", "cast", "country" columns by Unknown

df[["director", "cast", "country"]]=df[["director", "cast", "country"]].fillna("Unknown")

In [304]:
df.isnull().sum()

show_id          0
type             0
title            0
director         0
cast             0
country          0
date_added      10
release_year     0
rating           4
duration         3
listed_in        0
description      0
dtype: int64

In [305]:
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


## Clean `duration`

In [306]:
# separate measure of time and the value in a different columns
df["duration_measure"] = df["duration"].str.split(" ", expand = True, n= 0).get(1)
df["duration"] = df["duration"].str.split(" ", expand = True, n= 0).get(0)

In [307]:
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_measure
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,90,Documentaries,"As her father nears the end of his life, filmm...",min
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Seasons


In [308]:
df.columns 

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'duration_measure'],
      dtype='object')

In [309]:
# change the order of the columns
new_order = ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration','duration_measure', 'listed_in', 'description',
       ]


df = df.reindex(columns=new_order)
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,90,min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2,Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


In [310]:
# for the duration nulls, I will replace it  with the median, depending on if the row is a Movie or a TV Show
# first of all we need to check where we have the null values. Remember that we have only three null values
df[df["duration"].isnull()]



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,74 min,NaN,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,84 min,NaN,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,66 min,NaN,NaN,Movies,The comic puts his trademark hilarious/thought...


In [311]:
# in this case we have only null values in "Movies", so we need to calculate the median duration for this type of information
median_movies = df[df["type"] == "Movie"]["duration"].median()

for i, row in df.iterrows():
    if row["type"] == "Movie":
        df.loc[i, "duration"] = median_movies

In [312]:
# check the null values again
df.isnull().sum()

show_id              0
type                 0
title                0
director             0
cast                 0
country              0
date_added          10
release_year         0
rating               4
duration             0
duration_measure     3
listed_in            0
description          0
dtype: int64

## Clean `duration_measure`

In [313]:
# replace the null values of duration measure by "min" because all the missing values correspond to "movies"
df["duration_measure"]=df["duration_measure"].fillna("min")

In [314]:
# check the null values again
df.isnull().sum()

show_id              0
type                 0
title                0
director             0
cast                 0
country              0
date_added          10
release_year         0
rating               4
duration             0
duration_measure     0
listed_in            0
description          0
dtype: int64

# Clean `date_added`

In [315]:
df.isnull().sum()

show_id              0
type                 0
title                0
director             0
cast                 0
country              0
date_added          10
release_year         0
rating               4
duration             0
duration_measure     0
listed_in            0
description          0
dtype: int64

In [316]:
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,98.0,min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2,Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


In [317]:
print(list(df["date_added"].unique()))

['September 25, 2021', 'September 24, 2021', 'September 23, 2021', 'September 22, 2021', 'September 21, 2021', 'September 20, 2021', 'September 19, 2021', 'September 17, 2021', 'September 16, 2021', 'September 15, 2021', 'September 14, 2021', 'September 11, 2021', 'September 10, 2021', 'September 9, 2021', 'September 8, 2021', 'September 7, 2021', 'September 6, 2021', 'September 5, 2021', 'September 4, 2021', 'September 3, 2021', 'September 2, 2021', 'September 1, 2021', 'August 31, 2021', 'August 29, 2021', 'August 28, 2021', 'August 27, 2021', 'August 26, 2021', 'August 25, 2021', 'August 24, 2021', 'August 23, 2021', 'August 21, 2021', 'August 20, 2021', 'August 19, 2021', 'August 18, 2021', 'August 16, 2021', 'August 15, 2021', 'August 13, 2021', 'August 12, 2021', 'August 11, 2021', 'August 10, 2021', 'August 9, 2021', 'August 8, 2021', 'August 7, 2021', 'August 6, 2021', 'August 5, 2021', 'August 4, 2021', 'August 3, 2021', 'August 1, 2021', 'July 31, 2021', 'July 30, 2021', 'Jul

In [318]:
# we have the same structure in all the data "Month day, year". The usefull information in Month and year. So we try to get this information

df["year_added"] = df["date_added"].apply(lambda x: x.split(",")[1] if type(x) == str  else x) 
df["month_added"] = df["date_added"].apply(lambda x: x.split(" ")[0] if type(x) == str  else x) 

In [319]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,98.0,min,Documentaries,"As her father nears the end of his life, filmm...",2021,September
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2,Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021,September
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,"September 24, 2021",2021,TV-MA,1,Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021,September
3,s4,TV Show,Jailbirds New Orleans,Unknown,Unknown,Unknown,"September 24, 2021",2021,TV-MA,1,Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021,September
4,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2,Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021,September


## Clean `rating`

In [320]:
df["rating"].describe()

count      8803
unique       17
top       TV-MA
freq       3207
Name: rating, dtype: object

In [321]:
df[df["rating"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description,year_added,month_added
5989,s5990,Movie,13TH: A Conversation with Oprah Winfrey & Ava ...,Unknown,"Oprah Winfrey, Ava DuVernay",Unknown,"January 26, 2017",2017,NaN,98.0,min,Movies,Oprah Winfrey sits down with director Ava DuVe...,2017,January
6827,s6828,TV Show,Gargantia on the Verdurous Planet,Unknown,"Kaito Ishikawa, Hisako Kanemoto, Ai Kayano, Ka...",Japan,"December 1, 2016",2013,NaN,1,Season,"Anime Series, International TV Shows","After falling through a wormhole, a space-dwel...",2016,December
7312,s7313,TV Show,Little Lunch,Unknown,"Flynn Curry, Olivia Deeble, Madison Lu, Oisín ...",Australia,"February 1, 2018",2015,NaN,1,Season,"Kids' TV, TV Comedies","Adopting a child's perspective, this show take...",2018,February
7537,s7538,Movie,My Honor Was Loyalty,Alessandro Pepe,"Leone Frisa, Paolo Vaccarino, Francesco Miglio...",Italy,"March 1, 2017",2015,NaN,98.0,min,Dramas,"Amid the chaos and horror of World War II, a c...",2017,March


In this case, we have few null we don't remove it. 

## Clean `cast` 


In [322]:
# we need to separate each actor of the films
df['cast'] = df['cast'].apply(lambda x: x.split(','))
df = df.explode("cast")

## Clean `listed_in` 


In [323]:
# the same procedure 

df['listed_in'] = df['listed_in'].apply(lambda x: x.split(','))
df = df.explode("listed_in")

# Visualizations:

In [324]:
# How many Movies and TV Show we have?
type_ = df.groupby(['type']).size().reset_index(name='counts')
type_

,type,counts
0,Movie,101692
1,TV Show,47820


In [325]:
fig1 = px.pie(type_, values='counts', names='type', 
                  title='Distribution of Content Types on Netflix',
                  color_discrete_sequence=px.colors.qualitative.Set3)
fig1.show()

In [326]:
# how many films we have per raiting
ratings = df.groupby(['rating']).size().reset_index(name='counts')
ratings

,rating,counts
0,66 min,1
1,74 min,1
2,84 min,1
3,G,728
4,NC-17,71
5,NR,1133
6,PG,5955
7,PG-13,9860
8,R,15152
9,TV-14,38644


In [327]:

fig2 = px.pie(ratings, values='counts', names='rating', 
                  title='Distribution of Content Ratings on Netflix',
                  color_discrete_sequence=px.colors.qualitative.Set3)
fig2.show()

In [328]:
# top 5 successful directors on this platform


directors=df.groupby(['director']).size().reset_index(name='number_films').sort_values(by = "number_films", ascending = False)
directors.head(10)

,director,number_films
4303,Unknown,44621
685,Cathy Garcia-Molina,356
4481,Youssef Chahine,288
2671,Martin Scorsese,273
973,David Dhawan,270
4020,Steven Spielberg,243
2325,Kunle Afolayan,223
4258,Toshiya Shinohara,204
2522,Mae Czarina Cruz,198
4492,Yılmaz Erdoğan,197


In [329]:
# ignoring the "Unknown" directors, we select only the 5 most productive directors

top5_directors = directors[(directors["number_films"] >= directors.iloc[6, 1]  ) & (directors["number_films"] < directors.iloc[0, 1] )]
top5_directors

,director,number_films
685,Cathy Garcia-Molina,356
4481,Youssef Chahine,288
2671,Martin Scorsese,273
973,David Dhawan,270
4020,Steven Spielberg,243
2325,Kunle Afolayan,223


In [330]:
fig3=px.bar(top5_directors,x='number_films',y='director',title='Top 5 Directors on Netflix')
fig3.show()

In [331]:
# top 5 successful actors
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,98.0,min,Documentaries,"As her father nears the end of his life, filmm...",2021,September


In [332]:
actors=df.groupby(['cast']).size().reset_index(name='number_films').sort_values(by = "number_films", ascending = False)
actors.head(10)

,cast,number_films
38982,Unknown,1504
2612,Anupam Kher,109
38445,Shah Rukh Khan,74
30303,Takahiro Sakurai,70
23956,Paresh Rawal,70
23624,Om Puri,69
4186,Boman Irani,68
34037,Akshay Kumar,67
33367,Yuki Kaji,64
22504,Naseeruddin Shah,60


In [333]:
actors.iloc[0, 1]

1504

In [334]:
top5_actors = actors[(actors["number_films"] >= actors.iloc[6, 1] ) & (actors["number_films"] < actors.iloc[0, 1] )]
top5_actors

,cast,number_films
2612,Anupam Kher,109
38445,Shah Rukh Khan,74
30303,Takahiro Sakurai,70
23956,Paresh Rawal,70
23624,Om Puri,69
4186,Boman Irani,68


In [335]:
fig4=px.bar(top5_actors,x='number_films',y='cast',title='Top 5 Directors on Netflix')
fig4.show()

In [ ]:
class Sentiment: 
    def tokenize (string):
        try:
            tokenizer = RegexpTokenizer(r"\w+")
            tokens = tokenizer.tokenize(string)
            return tokens
        except: 
            return np.nan

        
    def stop_words(lista):
        stop_words = stopwords.words("english")
    
        nueva_lista = []
        
        for string in lista: 
            if string.lower() not in stop_words:
                nueva_lista.append(string)
        return " ".join(nueva_lista)

    def sentimientos(cancion):
        sia = SentimentIntensityAnalyzer()
        polarity = sia.polarity_scores(cancion)
        
        pol = polarity["compound"]
        return pol
        


In [ ]:
sen = Sentiment()

In [340]:
df["letras_tok"] = df["description"].apply(sen.tokenize)


In [342]:
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description,year_added,month_added,Sentiment,letras_tok
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,98.0,min,Documentaries,"As her father nears the end of his life, filmm...",2021,September,Positive,"[As, her, father, nears, the, end, of, his, li..."


In [344]:
df["letras_tok"] = df["letras_tok"].apply(sen.stop_words)


In [345]:
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_measure,listed_in,description,year_added,month_added,Sentiment,letras_tok
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,98.0,min,Documentaries,"As her father nears the end of his life, filmm...",2021,September,Positive,father nears end life filmmaker Kirsten Johnso...


In [346]:
# Downloading the vader lexicon from the nltk library.
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/anagarciagarcia/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [348]:
df["sentiment"] = df["letras_tok"].apply(sen.sentimientos)
